In [1]:
import skimage as sk
from skimage.io import imread, imshow, imsave
import numpy as np
import os
from glob import glob
import napari
from cellpose import models, io, utils
import pandas as pd
import pyclesperanto as cle
from imaris_ims_file_reader.ims import ims
from tqdm import tqdm

In [2]:
ims_files = sorted(glob('E:/ParadaKusz_Lab/OIC-153_Fluorescence-quantification-in-HUVEC-cells/H2DCFDA_05292025/**/*.ims',recursive=True))
imgs = [ims(file,ResolutionLevelLock=0,write=True) for file in ims_files]


Opening writeable file: E:/ParadaKusz_Lab/OIC-153_Fluorescence-quantification-in-HUVEC-cells/H2DCFDA_05292025\H2DCFA treatment assay_2025-05-29_Field_B10\H2DCFA treatment assay_2025-05-29_F028.ims 

Opening writeable file: E:/ParadaKusz_Lab/OIC-153_Fluorescence-quantification-in-HUVEC-cells/H2DCFDA_05292025\H2DCFA treatment assay_2025-05-29_Field_B10\H2DCFA treatment assay_2025-05-29_F029.ims 

Opening writeable file: E:/ParadaKusz_Lab/OIC-153_Fluorescence-quantification-in-HUVEC-cells/H2DCFDA_05292025\H2DCFA treatment assay_2025-05-29_Field_B10\H2DCFA treatment assay_2025-05-29_F030.ims 

Opening writeable file: E:/ParadaKusz_Lab/OIC-153_Fluorescence-quantification-in-HUVEC-cells/H2DCFDA_05292025\H2DCFA treatment assay_2025-05-29_Field_B10\H2DCFA treatment assay_2025-05-29_F031.ims 

Opening writeable file: E:/ParadaKusz_Lab/OIC-153_Fluorescence-quantification-in-HUVEC-cells/H2DCFDA_05292025\H2DCFA treatment assay_2025-05-29_Field_B11\H2DCFA treatment assay_2025-05-29_F032.ims 

Openi

c:\ProgramData\anaconda3\envs\CellPoseCLE-env\lib\site-packages\imaris_ims_file_reader\ims.py:189: UserWarning: swmr=True only affects read ('r') mode. For swmr write mode, set f.swmr_mode = True after opening the file.
  self.hf = h5py.File(self.filePathComplete, 'a', swmr=True)


Opening writeable file: E:/ParadaKusz_Lab/OIC-153_Fluorescence-quantification-in-HUVEC-cells/H2DCFDA_05292025\H2DCFA treatment assay_2025-05-29_Field_C3\H2DCFA treatment assay_2025-05-29_F194.ims 

Opening writeable file: E:/ParadaKusz_Lab/OIC-153_Fluorescence-quantification-in-HUVEC-cells/H2DCFDA_05292025\H2DCFA treatment assay_2025-05-29_Field_C3\H2DCFA treatment assay_2025-05-29_F195.ims 

Opening writeable file: E:/ParadaKusz_Lab/OIC-153_Fluorescence-quantification-in-HUVEC-cells/H2DCFDA_05292025\H2DCFA treatment assay_2025-05-29_Field_C4\H2DCFA treatment assay_2025-05-29_F196.ims 

Opening writeable file: E:/ParadaKusz_Lab/OIC-153_Fluorescence-quantification-in-HUVEC-cells/H2DCFDA_05292025\H2DCFA treatment assay_2025-05-29_Field_C4\H2DCFA treatment assay_2025-05-29_F197.ims 

Opening writeable file: E:/ParadaKusz_Lab/OIC-153_Fluorescence-quantification-in-HUVEC-cells/H2DCFDA_05292025\H2DCFA treatment assay_2025-05-29_Field_C4\H2DCFA treatment assay_2025-05-29_F198.ims 

Opening wr

In [3]:
all_img = [imgs[i][0,0,:,:,:] for i in range(len(imgs))]

In [42]:
import random
nums = range(len(all_img))
i = random.randint(min(nums),max(nums))
print(i)

129


In [43]:
test_image = all_img[i]
test_image.shape

(2048, 2048)

In [6]:
viewer = napari.view_image(test_image)

In [8]:
from cellpose import denoise

In [47]:
dn = denoise.DenoiseModel(model_type="denoise_cyto3", gpu=True)
imgs_dn = dn.eval(all_img, channels=None, diameter=50.)
imgs_dn = [np.squeeze(img) for img in imgs_dn]

2025-07-23 17:05:54,589 [INFO] >> denoise_cyto3 << model set to be used
2025-07-23 17:05:54,590 [INFO] ** TORCH CUDA version installed and working. **
2025-07-23 17:05:54,592 [INFO] >>>> using GPU (CUDA)
2025-07-23 17:05:54,975 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2025-07-23 17:07:33,457 [INFO] 100%|##########| 228/228 [01:38<00:00,  2.32it/s]


In [10]:
imgs_dn.shape

(2048, 2048, 1)

In [52]:
def normalize_images(input_image,tophat_radius):
    input_gpu = cle.push(input_image)
    #normalizing the image stack
    equalized_intensities_stack = cle.create_like(input_gpu)
    a_slice = cle.create([input_gpu.shape[1], input_gpu.shape[2]])
    num_slices = input_gpu.shape[0]
    mean_intensity_stack = cle.mean_of_all_pixels(input_gpu)
    corrected_slice = None
    for z in range(0, num_slices):
        # get a single slice out of the stack
        cle.copy_slice(input_gpu, a_slice, z)
        # measure its intensity
        mean_intensity_slice = cle.mean_of_all_pixels(a_slice)
        # correct the intensity
        correction_factor = mean_intensity_slice/mean_intensity_stack
        corrected_slice = cle.multiply_image_and_scalar(a_slice, corrected_slice, correction_factor)
        # copy slice back in a stack
        cle.copy_slice(corrected_slice, equalized_intensities_stack, z)
    #background subtraction (increase the signal to noise ratio for improved segmentation results)
    background_subtracted_top_hat = cle.top_hat_sphere(equalized_intensities_stack,radius_x=tophat_radius,radius_y=tophat_radius,radius_z=tophat_radius)
    #pull data off gpu
    input_pull = cle.pull(input_gpu)
    background_subtracted_top_hat_pull = cle.pull(background_subtracted_top_hat)
    equalized_intensities_stack_pull = cle.pull(equalized_intensities_stack)
    return background_subtracted_top_hat_pull

In [53]:
cle.select_device("NVIDIA")

(OpenCL) NVIDIA RTX A4000 (OpenCL 3.0 CUDA)
	Vendor:                      NVIDIA Corporation
	Driver Version:              573.24
	Device Type:                 GPU
	Compute Units:               48
	Global Memory Size:          16375 MB
	Local Memory Size:           0 MB
	Maximum Buffer Size:         4093 MB
	Max Clock Frequency:         1560 MHz
	Image Support:               Yes

In [48]:
io.logger_setup()
model = models.CellposeModel(gpu=True,model_type="cyto3") # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, _, _ = model.eval(imgs_dn, channels=channels, flow_threshold=0.4,cellprob_threshold=0.0, diameter=60)


creating new log file
2025-07-23 17:08:10,848 [INFO] WRITING LOG OUTPUT TO C:\Users\kristin.gallik\.cellpose\run.log
2025-07-23 17:08:10,849 [INFO] 
cellpose version: 	3.1.1.2 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.7.0+cu118
2025-07-23 17:08:10,851 [INFO] >> cyto3 << model set to be used
2025-07-23 17:08:10,854 [INFO] ** TORCH CUDA version installed and working. **
2025-07-23 17:08:10,856 [INFO] >>>> using GPU (CUDA)
2025-07-23 17:08:10,978 [INFO] >>>> loading model C:\Users\kristin.gallik\.cellpose\models\cyto3
2025-07-23 17:08:11,231 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2025-07-23 17:15:33,806 [INFO] 100%|##########| 228/228 [07:22<00:00,  1.94s/it]


In [50]:
imgs_stack = np.stack(all_img)
dn_imgs_stack = np.stack(imgs_dn)
masks_stack = np.stack(masks)

In [51]:
viewer = napari.view_image(imgs_stack,name='dn')
viewer.add_image(dn_imgs_stack,name='denoise')
viewer.add_labels(masks_stack,name='masks')

<Labels layer 'masks' at 0x1a82e9bf8b0>

In [54]:
corrected = normalize_images(imgs_stack,tophat_radius=10)

In [ ]:
io.logger_setup()
model = models.CellposeModel(gpu=True,model_type="cyto3") # model_type='cyto' or 'nuclei' or 'cyto2' or 'cyto3'
channels =[[0,0]]
masks, _, _ = model.eval(corrected, channels=channels, flow_threshold=0.4,cellprob_threshold=0.0, diameter=60)


creating new log file
2025-07-23 17:34:09,302 [INFO] WRITING LOG OUTPUT TO C:\Users\kristin.gallik\.cellpose\run.log
2025-07-23 17:34:09,302 [INFO] 
cellpose version: 	3.1.1.2 
platform:       	win32 
python version: 	3.10.16 
torch version:  	2.7.0+cu118
2025-07-23 17:34:09,302 [INFO] >> cyto3 << model set to be used
2025-07-23 17:34:09,307 [INFO] ** TORCH CUDA version installed and working. **
2025-07-23 17:34:09,309 [INFO] >>>> using GPU (CUDA)
2025-07-23 17:34:09,462 [INFO] >>>> loading model C:\Users\kristin.gallik\.cellpose\models\cyto3
2025-07-23 17:34:09,679 [INFO] >>>> model diam_mean =  30.000 (ROIs rescaled to this size during training)
2025-07-23 17:34:09,683 [WARNING] z_axis not specified, assuming it is dim 0
2025-07-23 17:34:09,684 [WARNING] if this is actually the channel_axis, use 'model.eval(channel_axis=0, ...)'
2025-07-23 17:34:09,686 [INFO] multi-stack tiff read in as having 228 planes 1 channels
2025-07-23 17:34:14,730 [WARNING] normalize_params['norm3D'] is True 

In [ ]:
masks_stack = np.stack(masks)

In [57]:
viewer = napari.view_image(corrected)
viewer.add_labels(masks, name='masks')

<Labels layer 'masks' at 0x1a8800cbfa0>

In [58]:
props = sk.measure.regionprops_table(masks,properties=('label','area'))

In [59]:
props

{'label': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
        35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
        52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
        69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
        86]),
 'area': array([671727., 606616., 737498., 695428., 697457., 693110., 642178.,
        698324., 685421., 669581., 715415., 732500., 742166., 679192.,
        774457., 687111., 768060., 719776., 719739., 722366., 726849.,
        722498., 664574., 639418., 647638., 675031., 591554., 602809.,
        618768., 605889., 507183., 538302., 483042., 506347., 511041.,
        475011., 405220., 405826., 398947., 341527., 373755., 339460.,
        327827., 334443., 318082., 312796., 258046., 275991., 297844.,
        237863., 244225., 206262., 184495., 172328., 136048., 136533.,
        107788., 1084